# 👨‍💻🔍 EDA Topics des Articles

Ce notebook permet d'explorer les possibilités liées à la tâche Notion : Classifier le topic des articles afin de pouvoir comparer les sujets.

In [13]:
import sys
sys.path.append('..')
from quotaclimat.data_processing.sitemap.sitemap_processing import load_all

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

In [4]:
# Data import and check
df = load_all("../data_public/sitemap_dumps/")
print(df.shape)
print(df.columns)
df.head()

(320545, 26)
Index(['url', 'news', 'news_publication', 'publication_name',
       'publication_language', 'news_publication_date', 'news_title',
       'news_keywords', 'image', 'image_loc', 'image_caption', 'sitemap',
       'etag', 'sitemap_last_modified', 'sitemap_size_mb', 'download_date',
       'media', 'section', 'changefreq', 'news_access', 'image_title',
       'lastmod', 'news_genres', 'priority', 'download_date_last',
       'media_type'],
      dtype='object')


,url,news,news_publication,publication_name,publication_language,news_publication_date,news_title,news_keywords,image,image_loc,image_caption,sitemap,etag,sitemap_last_modified,sitemap_size_mb,download_date,media,section,changefreq,news_access,image_title,lastmod,news_genres,priority,download_date_last,media_type
37884,https://www.francetvinfo.fr/monde/nouvel-an-20...,\n,\n,Franceinfo,fr,2022-12-31 15:57:46,Nouvel An 2023 : le feu d’artifice fait son re...,"Nouvel An 2023, Monde, France",\n,https://www.francetvinfo.fr/image/761i6v10w-6c...,Nouvel An 2023 : le feu d’artifice fait son re...,https://www.francetvinfo.fr/sitemap_news.xml,"""059a07258cabf77ded25d09acd8d9ff3d7e01e84""",2023-01-01 01:46:04+00:00,0.216074,2023-01-01 01:48:02,francetvinfo,"[monde, nouvel-an-2021]",None,None,None,None,None,NaN,2023-01-02 01:28:01,tv
37885,https://www.francetvinfo.fr/replay-jt/france-3...,\n,\n,Franceinfo,fr,2022-12-31 16:03:28,"Nouvel An 2023 : dans la Marne, des éthylotest...","Nouvel An 2023, Monde, Sécurité routière, Mode...",\n,https://www.francetvinfo.fr/image/761i6v4km-1a...,"Nouvel An 2023 : dans la Marne, des éthylotest...",https://www.francetvinfo.fr/sitemap_news.xml,"""059a07258cabf77ded25d09acd8d9ff3d7e01e84""",2023-01-01 01:46:04+00:00,0.216074,2023-01-01 01:48:02,francetvinfo,"[replay-jt, france-3, 12-13]",None,None,None,None,None,NaN,2023-01-02 01:28:01,tv
37886,https://www.francetvinfo.fr/culture/charente-m...,\n,\n,Franceinfo,fr,2022-12-31 16:05:38,Charente-Maritime : découverte de la ferme des...,"Culture, Animaux",\n,https://www.francetvinfo.fr/image/761i6v4sd-70...,Charente-Maritime : découverte de la ferme des...,https://www.francetvinfo.fr/sitemap_news.xml,"""059a07258cabf77ded25d09acd8d9ff3d7e01e84""",2023-01-01 01:46:04+00:00,0.216074,2023-01-01 01:48:02,francetvinfo,[culture],None,None,None,None,None,NaN,2023-01-02 01:28:01,tv
37887,https://www.francetvinfo.fr/monde/nouvel-an-20...,\n,\n,Franceinfo,fr,2022-12-31 16:00:23,Nouvel An 2023 : un dispositif de sécurité imp...,"Nouvel An 2023, Monde",\n,https://www.francetvinfo.fr/image/761i6v19q-42...,Nouvel An 2023 : un dispositif de sécurité imp...,https://www.francetvinfo.fr/sitemap_news.xml,"""059a07258cabf77ded25d09acd8d9ff3d7e01e84""",2023-01-01 01:46:04+00:00,0.216074,2023-01-01 01:48:02,francetvinfo,"[monde, nouvel-an-2021]",None,None,None,None,None,NaN,2023-01-02 01:28:01,tv
37888,https://www.francetvinfo.fr/replay-radio/micro...,\n,\n,Franceinfo,fr,2022-12-31 15:42:02,La fondation René Cassin : développer les droi...,"Europe, Mode de vie, lifestyle, Formation, éco...",\n,https://www.francetvinfo.fr/image/761i6wgbd-5e...,"Le juriste, diplomate et homme politique franç...",https://www.francetvinfo.fr/sitemap_news.xml,"""059a07258cabf77ded25d09acd8d9ff3d7e01e84""",2023-01-01 01:46:04+00:00,0.216074,2023-01-01 01:48:02,francetvinfo,"[replay-radio, micro-europeen]",None,None,None,None,None,NaN,2023-01-02 01:28:01,tv


In [6]:
df_title = df[['news_title']]
df_title

,news_title
37884,Nouvel An 2023 : le feu d’artifice fait son re...
37885,"Nouvel An 2023 : dans la Marne, des éthylotest..."
37886,Charente-Maritime : découverte de la ferme des...
37887,Nouvel An 2023 : un dispositif de sécurité imp...
37888,La fondation René Cassin : développer les droi...
...,...
87145,Le pic de la 9e vague de Covid-19 désormais fr...
87146,Qui est mon père Noël ?
87147,"Ce soir à la télé : : « Ben-Hur », le grand ba..."
87148,"Interdites de travailler avec des Afghanes, de..."


In [19]:
# Preprocess data
vectorizer = CountVectorizer(stop_words=list(fr_stop))
X = vectorizer.fit_transform(df_title['news_title'])

# Topic modeling with LDA
n_topics = 10
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)

/Users/9204057K/.pyenv/versions/3.10.2/envs/quotaclimat/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


LatentDirichletAllocation(random_state=42)

In [20]:
# Print the top words for each topic
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-15 - 1:-1]]
    print("Topic %d: %s" % (topic_idx, ", ".join(top_words)))

Topic 0: prix, stations, meilleurs, aujourd, essence, saint, motion, censure, mer, pont, rance, lampaul, roche, sainte, pommerit
Topic 1: ans, prison, homme, condamné, contre, affaire, procès, ski, unis, violences, enquête, nice, ex, lyon, mort
Topic 2: turquie, météo, syrie, séisme, rhône, morts, ans, iran, nord, alpes, brest, hôpital, sud, pays, noël
Topic 3: ukraine, guerre, 2022, prix, inflation, hausse, russie, euros, décembre, 01, unis, face, 12, 2023, crise
Topic 4: saint, 2023, école, avril, mars, conseil, février, météo, prévisions, élèves, samedi, dimanche, janvier, pierre, mer
Topic 5: ligue, club, tennis, foot, marseille, sainte, uni, royaume, psg, match, heure, soline, seine, darmanin, contre
Topic 6: retraites, réforme, contre, mobilisation, week, journée, end, france, manifestants, football, faut, qu, incendie, grève, 000
Topic 7: retraites, macron, réforme, emmanuel, borne, gouvernement, olivier, eau, alpes, loi, projet, qu, veut, ministre, assemblée
Topic 8: paris, grè

In [21]:
# Assign topics to new titles
new_titles = df_title['news_title'].copy()
X_new = vectorizer.transform(new_titles)
topic_probs = lda.transform(X_new)
topic_labels = topic_probs.argmax(axis=1)

# Add topic labels to the original data frame
df_title['topic'] = topic_labels

df_title.head()

/var/folders/x5/48zrqp5j59s3dl40qn9shksr0000gp/T/ipykernel_33533/522455191.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_title['topic'] = topic_labels


,news_title,topic
37884,Nouvel An 2023 : le feu d’artifice fait son re...,6
37885,"Nouvel An 2023 : dans la Marne, des éthylotest...",8
37886,Charente-Maritime : découverte de la ferme des...,1
37887,Nouvel An 2023 : un dispositif de sécurité imp...,8
37888,La fondation René Cassin : développer les droi...,4
